In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import f


from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller, arma_order_select_ic

import warnings
warnings.filterwarnings("ignore")

In [64]:
#LOADING DATA SET

df = pd.read_csv("CTA_data.csv")
ts = df[['service_date','total_rides']]
ts['date'] = pd.to_datetime(ts['service_date'])
ts.drop(columns = ['service_date'], axis = 1, inplace = True)
ts.set_index('date', inplace = True)

In [65]:
ts_M = ts.resample('M').mean()

In [66]:
#f-test
def f_test(a,b,c):
    f_stat = (a-b/4)/(b/210-(4*c+4))
    pvalue = f.cdf(f_stat,4,210-(4*c+4))
    
    return 1-pvalue
    

#MSE function
def mse(res):
    error = 0
    for r in res:
        error += r**2
    return error

err = [0]

#test function
for n in range(1,5):
    model = ARMA(ts_M, order=(2*n,2*n-1))
    results = model.fit(trend='nc', method='mle')
    
    err.append(mse(results.resid))
    
    if(n>1):
        value = f_test(err[n-1],err[n],n)
        if(value > 0.05):
            break
            
            
            
print("Best model is ARMA(%s,%s)"%(2*(n-1),2*(n-1)-1))
    

Best model is ARMA(6,5)
